**This credit score predictor uses Kaggle's "give me some credit dataset" for predicting the credit score of the client**
**the predictor calculates what is the chance of the client actually returning the loan given to them,or whether the client will face some kind of financial crisis and won't be able to return the loan**






the first cell is mainly removing the the rows in the training data which contain any feature with NULL in it. as they may disturb the training of the network

In [0]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from IPython.display import display


# Pretty display for notebooks
%matplotlib inline 



data = pd.read_csv("cs-test.csv")

# Dropping the first unnamed column
data = data.drop(['id'], axis = 1)

#display the first 5 records
display(data.head(5))

#removing all the rows with null entries in them 
x=data
ind=x[x['MonthlyIncome'].isnull()].index.tolist()
x.drop(x.index[[ind]], inplace=True)
print('indexes')
print(x[x['MonthlyIncome'].isnull()].index.tolist())
print(x[x['NumberOfDependents'].isnull()].index.tolist())
x.to_csv('test.csv',index=False)


the second cell is dividing the training data into features and label file.

In [0]:
import csv
j=[]
pro=open('feature.csv','a')
with open('x.csv', 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile ,delimiter=',')
    print(csvreader)
    for row in csvreader:
        
        for i in row[1:]:
            pro.write(i+",")
        pro.write("\n")    
    pro.close()

j=[]
pro=open('label.csv','a')
with open('x.csv', 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile ,delimiter=',')
    print(csvreader)
    for row in csvreader:
        pro.write(row[0])
        pro.write("\n")    
    pro.close()


this cell is converting the features and label file into numpy arrays and dividing the data into training and test sets,and then scalling the data for normalisation.

In [1]:
import csv
import numpy as np
f_data_x=[]
with open('feature.csv', 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile ,delimiter=',')
    print(csvreader)
    data=list(csvreader)
    
    for i in data:
        f_data_x.append([float(k) for k in i[:-1]])

        

    
f_data_y=[]
with open('label.csv', 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile ,delimiter=',')
    print(csvreader)
    data=list(csvreader)
    
    for i in data:
      #print(i)
      f_data_y.append(float(i[0]))

        
train_x=np.array(f_data_x[0:110000],dtype=float)
train_y=np.array(f_data_y[0:110000],dtype=float)
test_x=np.array(f_data_x[110000:len(f_data_x)],dtype=float)
test_y=np.array(f_data_y[110000:len(f_data_x)],dtype=float)

train_x=train_x/train_x.max(axis=0)
test_x=test_x/test_x.max(axis=0)




This cell is creating the neural network and and training it with the training data that we got after seprating the available data into training and test sets.

In [3]:
import tensorflow as tf
class mycallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if logs.get('acc')>0.99:
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training=True
      
      
  

callback=mycallback()  



model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10,input_shape=(10,),activation=tf.nn.relu),
    
    tf.keras.layers.Dense(8,activation=tf.nn.relu),

    tf.keras.layers.Dense(8,activation=tf.nn.relu),

    tf.keras.layers.Dense(8,activation=tf.nn.relu),

    tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)
   
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_x,train_y,epochs=10, callbacks=[callback])


Epoch 1/10
110000/110000 [==============================] - 14s 127us/sample - loss: 0.2120 - acc: 0.9313
Epoch 2/10
110000/110000 [==============================] - 14s 127us/sample - loss: 0.2035 - acc: 0.9338
Epoch 3/10
110000/110000 [==============================] - 14s 129us/sample - loss: 0.2019 - acc: 0.9339
Epoch 4/10
110000/110000 [==============================] - 14s 128us/sample - loss: 0.2018 - acc: 0.9342
Epoch 5/10
110000/110000 [==============================] - 14s 127us/sample - loss: 0.2011 - acc: 0.9339
Epoch 6/10
110000/110000 [==============================] - 14s 130us/sample - loss: 0.2010 - acc: 0.9342
Epoch 7/10
110000/110000 [==============================] - 14s 129us/sample - loss: 0.2013 - acc: 0.9344
Epoch 8/10
110000/110000 [==============================] - 14s 130us/sample - loss: 0.2009 - acc: 0.9340
Epoch 9/10
110000/110000 [==============================] - 14s 128us/sample - loss: 0.2010 - acc: 0.9339
Epoch 10/10
110000/110000 [===================

Saving the model for future use.

In [0]:
model.save("creditpredictor.h5")

evaluating the model on test data

In [5]:
model.evaluate(test_x,test_y)

10269/10269 [==============================] - 1s 61us/sample - loss: 0.2219 - acc: 0.9301


[0.22192176962876253, 0.93008083]

calculating the credit score on any test example , simply specify the example number between 1 and 10000

In [6]:
score=model.predict(np.array([train_x[0]]))
#print(score)
print("credit score of mentioned example is" ,1000-int(score[0][0]*1000))

credit score of mentioned example is 749


In [8]:
newmodel=tf.keras.models.load_model("creditpredictor.h5")
score=newmodel.predict(np.array([train_x[0]]))
#print(score)
print("credit score of mentioned example is" ,1000-int(score[0][0]*1000))

credit score of mentioned example is 749
